<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#AB-Testing" data-toc-modified-id="AB-Testing-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>AB-Testing</a></span></li></ul></div>

# AB-Testing

![cats](images/cats.jpeg)


Imaginad que somos los cientificos de datos de la empresa de videojuegos Tactile Entertainment. Los desarrolladores del juego Cookie Cats pretenden introducir un cambio en el juego para aumentar la retencion de los jugadores. En cierto nivel del juego los jugadores se encuentran una puerta que les obliga a esperar o a pagar la app. Actualmente la puerta se encuentra en nivel 30 y se pretende pasar al nivel 40, para comprobar la retencion a 1 y 7 dias. Antes de realizar el cambio definitivo en el juego se raliza un test AB.

Los datos estan alojados en `data/cookie_cats.csv`. Nuestro grupo de control sera la version actual `gate_30` y el grupo de tratamiento sera la version `gate_40`. Debemos realizar el test para 1 dia de retencion `retention_1` y para 7 dias `retention_7`.

In [28]:
# librerias

import pandas as pd
import numpy as np

from statsmodels.stats.proportion import proportions_ztest, proportion_confint
from scipy.stats import norm, sem

import statsmodels.stats.api as sms

import pylab as plt
from bayes import *
%matplotlib inline

In [29]:
# datos
df = pd.read_csv('data/cookie_cats.csv')
df.shape

(90189, 5)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [51]:
matrix1 = pd.crosstab(df.version, df.retention_1)
matrix1

retention_1,False,True
version,,
gate_30,24666,20034
gate_40,25370,20119


In [70]:
ret301 = 24666/(24666+20034)
ret401 = 25370/(25370+20119)
ret301, ret401, (ret401-ret301)/ret301


(0.5518120805369128, 0.5577172503242542, 0.010701414477181674)

In [32]:
matrix7 = pd.crosstab(df.version, df.retention_7)
matrix7

retention_7,False,True
version,,
gate_30,36198,8502
gate_40,37210,8279


In [85]:
matrix7[1][0]/matrix7[0][0], matrix7[1][1]/matrix7[0][1]


(0.23487485496436267, 0.22249395323837678)

In [33]:
# En el enunciado ya se nos indica como grupod de control el grupo gate_30 y el otro como grupo de tratamiento, 
# vamos a crearnos cuantro series con cada uno de los grupos (control/tratamiento) y con los dos estudios que deseamos realizar

control_res1=df[df.version=='gate_30']['retention_1']
tratamiento_res1=df[df.version=='gate_40']['retention_1']

control_res7=df[df.version=='gate_30']['retention_7']
tratamiento_res7=df[df.version=='gate_40']['retention_7']



In [119]:
# Para poder reutilizar codigo, voy a crearme una funcion a la que le entren las 2 series (control/tratamiento)
#  y devuelva z-score y pvalue,
def proportion_test(cont, trat):
    """ This function is meant to study a set of 2 Series, and it an it carries outan AB test with them,
    INPUTS: --------------------------------------------------------------------------------
            - cont --> Control Serie
            - trat --> Treatment Serie
    OUTPUTS:--------------------------------------------------------------------------------
            - (z_score, p_value)
    """
    conversiones=[cont.sum(), trat.sum()]  # conversiones por grupo

    z_score, p_value = proportions_ztest(conversiones, nobs=[cont.count(), trat.count()])

    return z_score, p_value

z1_score, p1_value = proportion_test(control_res1, tratamiento_res1)


In [37]:
# Ahora hacerlo para la retencion a 7 dias solo requiere una linia de codigo mas,
proportion_test(control_res7, tratamiento_res7)

(3.164358912748191, 0.001554249975614329)

In [87]:
# Con el estudio que hemos realizado hasta ahora ya podemos sacar algunas conclusiones
# Vamos a comentar primero los resultados para la retencion a 1 dia,

""" Si vemos una cierta tendencia al aumento en cuanto a la retencion al dia de descargarse la aplicación,
 Aun asi el pvalue es ligeramente superior a 0.05, lo que nos indica un rango de incertidumbre en cuanto a la mejoria mayor a un 5% 
 (alrededor de un 7.5%). Es decir, vemos una mejoria en la retencion de alrededor de un 1% y podemos afirmar con una seguridad del 93% que 
 esta mejoria se debe al cambio realizado y no es aleatorio."""

 # En el segundo resultado,
""" En este caso el pvalue disminuye a un 0.0016 lo que indica que la probabilidad de que el cambio observado se deba a la aleatoriedad
    es despreciable, el cambio observado se debe con mucha probabilidad al cambio aplicado. Lo que pasa esque vemos una disminucion en la retencion
     a los 7 dias. Es decir, la fraccion de gente que sigue jugando a los 7 dias es menor como conseqüencia de los cambios aplicados. """


' En este caso el pvalue disminuye a un 0.0016 lo que indica que la probabilidad de que el cambio observado se deba a la aleatoriedad\n    es despreciable, el cambio observado se debe con mucha probabilidad al cambio aplicado. Lo que pasa esque vemos una disminucion en la retencion\n     a los 7 dias. Es decir, la fraccion de gente que sigue jugando a los 7 dias es menor como conseqüencia de los cambios aplicados. '

In [98]:
# A partir de aqui podemos jugar un poco con los datos, y hacer otros estudios
# por ejemplo, vamos a targetear a la gente que ha jugado entre 20 y 60 partidas,
df_subgrup = df[(df.sum_gamerounds>20)&(df.sum_gamerounds<60)]
print(df_subgrup.shape) # Vemos que aunque se reduzca muchisimo nuestra muestra sigue siendo un grupo considerable
print(df_subgrup.version[df_subgrup.version == 'gate_30'].count(),df_subgrup.version[df_subgrup.version == 'gate_40'].count())
# vemos tambien que las muestras estudiadas siguen estando repartidas con porciones parecidas. 


(20448, 5)
10454 9994


In [106]:

# Vamos a repetir el estudio con este grupo,

control_res1=df_subgrup[df_subgrup.version=='gate_30']['retention_1']
tratamiento_res1=df_subgrup[df_subgrup.version=='gate_40']['retention_1']

control_res7=df_subgrup[df_subgrup.version=='gate_30']['retention_7']
tratamiento_res7=df_subgrup[df_subgrup.version=='gate_40']['retention_7']

print( proportion_test(control_res1, tratamiento_res1),';',  proportion_test(control_res7, tratamiento_res7))
ret730 = df_subgrup[df_subgrup.version=='gate_30']['retention_7'].sum()/df_subgrup[df_subgrup.version=='gate_30']['retention_7'].count()
ret740 = df_subgrup[df_subgrup.version=='gate_40']['retention_7'].sum()/df_subgrup[df_subgrup.version=='gate_40']['retention_7'].count()
print(ret730, ret740)

(0.34121252681544784, 0.7329435935358919) ; (3.076830228784151, 0.0020921442040044455)
0.17859192653529748 0.16239743846307786


In [108]:
# Voy a generar una funcion que nos estudio el proceso en el rango que le indiquemos,
def rang_ABtesting_CatGame(a=0,b=max(df.sum_gamerounds)):
    df_subgrup = df[(df.sum_gamerounds>a)&(df.sum_gamerounds<b)]
    control_res1=df_subgrup[df_subgrup.version=='gate_30']['retention_1']
    tratamiento_res1=df_subgrup[df_subgrup.version=='gate_40']['retention_1']

    control_res7=df_subgrup[df_subgrup.version=='gate_30']['retention_7']
    tratamiento_res7=df_subgrup[df_subgrup.version=='gate_40']['retention_7']
    return proportion_test(control_res1, tratamiento_res1),  proportion_test(control_res7, tratamiento_res7)

In [118]:
rang_ABtesting_CatGame()


((1.5805887864476007, 0.11397209143329877),
 (3.006086884574141, 0.002646333709773733))

49854